<a href="https://colab.research.google.com/github/isaacd68/MAT494-data-science/blob/main/1_3LinearAlgebra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***QR Decompostion***

To calculate the QR Decomposition of a matrix  with NumPy/SciPy, we can make use of the built-in linalg library via the linalg.qr function. This is significantly more efficient than using a pure Python implementation:

In [ ]:
import pprint
import numpy as np
import scipy.linalg   # SciPy Linear Algebra Library

A = np.array([[12, -51, 4], [6, 167, -68], [-4, 24, -41]])  
Q, R = scipy.linalg.qr(A)

print ("A:")
pprint.pprint(A)

print ("Q:")
pprint.pprint(Q)

print ("R:")
pprint.pprint(R)

A:
array([[ 12, -51,   4],
       [  6, 167, -68],
       [ -4,  24, -41]])
Q:
array([[-0.85714286,  0.39428571,  0.33142857],
       [-0.42857143, -0.90285714, -0.03428571],
       [ 0.28571429, -0.17142857,  0.94285714]])
R:
array([[ -14.,  -21.,   14.],
       [   0., -175.,   70.],
       [   0.,    0.,  -35.]])


The output of $R$ is a check to make sure it is an upper triangular matrix.

Using the Householder Reflections algorithim to have a better idea for QR decomposition in python.

In [ ]:
from math import sqrt
from pprint import pprint
import math
import numpy as np
import scipy.linalg

A = np.array([[12, -51, 4], [6, 167, -68], [-4, 24, -41]])  
Q, R = scipy.linalg.qr(A)

print ("A:")
pprint(A)

print ("Q:")
pprint(Q)

print ("R:")
pprint(R) 

def mult_matrix(M, N):
    """Multiply square matrices of same dimension M and N"""
    # Converts N into a list of tuples of columns                                                                     
    tuple_N = zip(*N)

    # Nested list comprehension to calculate matrix multiplication                                                    
    return [[sum(el_m * el_n for el_m, el_n in zip(row_m, col_n)) for col_n in tuple_N] for row_m in M]

def trans_matrix(M):
    """Take the transpose of a matrix."""
    n = len(M)
    return [[ M[i][j] for i in range(n)] for j in range(n)]

def norm(x):
    """Return the Euclidean norm of the vector x."""
    return sqrt(sum([x_i**2 for x_i in x]))

def Q_i(Q_min, i, j, k):
    """Construct the Q_t matrix by left-top padding the matrix Q                                                      
    with elements from the identity matrix."""
    if i < k or j < k:
        return float(i == j)
    else:
        return Q_min[i-k][j-k]

def householder(A):
    """Performs a Householder Reflections based QR Decomposition of the                                               
    matrix A. The function returns Q, an orthogonal matrix and R, an                                                  
    upper triangular matrix such that A = QR."""
    n = len(A)

    # Set R equal to A, and create Q as a zero matrix of the same size
    R = A
    Q = [[0.0] * n for i in range(n)]

    # The Householder procedure
    for k in range(n-1):  # We don't perform the procedure on a 1x1 matrix, so we reduce the index by 1
        # Create identity matrix of same size as A                                                                    
        I = [[float(i == j) for i in range(n)] for j in range(n)]
    
        # Create the vectors x, e and the scalar alpha
        ang = 0
        x = [row[k] for row in R[k:]]
        e = [row[k] for row in I[k:]]
        alpha = -math.copysign(x[0], ang) * norm(x)

        # Using anonymous functions, we create u and v
        u = map(lambda p,q: p + alpha * q, x, e)
        norm_u = norm(u)
        v = map(lambda p: p/norm_u, u)

        # Create the Q minor matrix
        Q_min = [[float(i==j) - 2.0 * v[i] * v[j] for i in range(n-k)] for j in range(n-k)]
        for i in range(n-k):
              print(i)
        for j in range(n-k):
              print(j) 
        # "Pad out" the Q minor matrix with elements from the identity
        Q_t = [[Q_i(Q_min,i,j,k) for i in range(n)] for j in range(n)]

        # If this is the first run through, right multiply by A,
        # else right multiply by Q
        if k == 0:
            Q = Q_t
            R = mult_matrix(Q_t,A)
        else:
            Q = mult_matrix(Q_t,Q)
            R = mult_matrix(Q_t,R)

    # Since Q is defined as the product of transposes of Q_t,
    # we need to take the transpose upon returning it
    return trans_matrix(Q), R

A = [[12, -51, 4], [6, 167, -68], [-4, 24, -41]]
Q, R = householder(A)

print ("A:")
pprint(A)

print ("Q:")
pprint(Q)

print ("R:")
pprint(R)

A:
array([[ 12, -51,   4],
       [  6, 167, -68],
       [ -4,  24, -41]])
Q:
array([[-0.85714286,  0.39428571,  0.33142857],
       [-0.42857143, -0.90285714, -0.03428571],
       [ 0.28571429, -0.17142857,  0.94285714]])
R:
array([[ -14.,  -21.,   14.],
       [   0., -175.,   70.],
       [   0.,    0.,  -35.]])


TypeError: ignored

***Least Squares in python***

Generate data using the scikit-learn library.

In [ ]:
X, y, coefficients = make_regression(
    n_samples=50,
    n_features=1,
    n_informative=1,
    n_targets=1,
    noise=5,
    coef=True,
    random_state=1
)

Store the the rank and the number of columns of the matrix as variables.

In [ ]:
n = X.shape[1]
r = np.linalg.matrix_rank(X)

Find the equivalent to our matrix of features using singular value decomposition

In [ ]:
U, sigma, VT = np.linalg.svd(X, full_matrices=False)

 $D^+$ can be derived from sigma

In [ ]:
D_plus = np.diag(np.hstack([1/sigma[:r], np.zeros(n-r)]))

V is of course equal to the transpose of its transpose as described in the following identity.

In [ ]:
V = VT.T

Determine Moore-Penrose pseudoinverse of X.

In [ ]:
X_plus = V.dot(D_plus).dot(U.T)

The vector of coefficients can be calculated by multiplying the pseudoinverse of the matrix X by y

In [ ]:
w = X_plus.dot(y)

The actual error, we compute the residual sum of squares using the very first equation we saw.

In [ ]:
error = np.linalg.norm(X.dot(w) - y, ord=2) ** 2

All together:

In [ ]:
from sklearn.datasets import make_regression
from matplotlib import pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression

X, y, coefficients = make_regression(
    n_samples=50,
    n_features=1,
    n_informative=1,
    n_targets=1,
    noise=5,
    coef=True,
    random_state=1
)

n = X.shape[1]
r = np.linalg.matrix_rank(X)

U, sigma, VT = np.linalg.svd(X, full_matrices=False)

D_plus = np.diag(np.hstack([1/sigma[:r], np.zeros(n-r)]))

V = VT.T

X_plus = V.dot(D_plus).dot(U.T)

w = X_plus.dot(y)

error = np.linalg.norm(X.dot(w) - y, ord=2) ** 2
print(error)

888.9637001387936


***Linear Regression in Python***

The second step is defining data to work with. The inputs (regressors, 𝑥) and output (predictor, 𝑦) should be arrays (the instances of the class numpy.ndarray) or similar objects. This is the simplest way of providing data for regression:

Now, you have two arrays: the input x and output y. You should call .reshape() on x because this array is required to be two-dimensional, or to be more precise, to have one column and as many rows as necessary. That’s exactly what the argument (-1, 1) of .reshape() specifies.

As you can see, x has two dimensions, and x.shape is (6, 1), while y has a single dimension, and y.shape is (6,).

Let’s create an instance of the class LinearRegression, which will represent the regression mode

This statement creates the variable model as the instance of LinearRegression. You can provide several optional parameters to LinearRegression

With .fit(), you calculate the optimal values of the weights 𝑏₀ and 𝑏₁, using the existing input and output (x and y) as the arguments. In other words, .fit() fits the model. It returns self, which is the variable model itself. That’s why you can replace the last two statements with this one

Once you have your model fitted, you can get the results to check whether the model works satisfactorily and interpret it.

You can obtain the coefficient of determination ($R^2$) with .score() called on model

When you’re applying .score(), the arguments are also the predictor x and regressor y, and the return value is $R^2$.

The attributes of model are .intercept_, which represents the coefficient, $b_0$ and .coef_, which represents $b_1$

The code above illustrates how to get $b_0$ and $b_1$. You can notice that .intercept_ is a scalar, while .coef_ is an array.

The value $b_0$ = 5.63 (approximately) illustrates that your model predicts the response 5.63 when $x$ is zero. The value $b_1$ = 0.54 means that the predicted response rises by 0.54 when $x$ is increased by one.

You should notice that you can provide y as a two-dimensional array as well. In this case, you’ll get a similar result. This is how it might look

this example is very similar to the previous one, but in this case, .intercept_ is a one-dimensional array with the single element $b_0$, and .coef_ is a two-dimensional array with the single element $b_1$

To obtain the predicted response, use .predict()

When applying .predict(), you pass the regressor as the argument and get the corresponding predicted response.

This is a nearly identical way to predict the response

In this case, you multiply each element of x with model.coef_ and add model.intercept_ to the product.

The output here differs from the previous example only in dimensions. The predicted response is now a two-dimensional array, while in the previous case, it had one dimension.

If you reduce the number of dimensions of x to one, these two approaches will yield the same result. You can do this by replacing x with x.reshape(-1), x.flatten(), or x.ravel() when multiplying it with model.coef_.

In practice, regression models are often applied for forecasts. This means that you can use fitted models to calculate the outputs based on some other, new inputs

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

x = np.array([5, 15, 25, 35, 45, 55]).reshape((-1, 1))
y = np.array([5, 20, 14, 32, 22, 38])
print(x)
print(y)

model = LinearRegression()

model.fit(x, y)

model = LinearRegression().fit(x, y)

r_sq = model.score(x, y)
print('coefficient of determination:', r_sq)
print('intercept:', model.intercept_)
print('slope:', model.coef_)

new_model = LinearRegression().fit(x, y.reshape((-1, 1)))
print('intercept:', new_model.intercept_)
print('slope:', new_model.coef_)

y_pred = model.predict(x)
print('predicted response:', y_pred, sep='\n')

y_pred = model.intercept_ + model.coef_ * x
print('predicted response:', y_pred, sep='\n')

x_new = np.arange(5).reshape((-1, 1))
print(x_new)
y_new = model.predict(x_new)
print(y_new)

[[ 5]
 [15]
 [25]
 [35]
 [45]
 [55]]
[ 5 20 14 32 22 38]
coefficient of determination: 0.7158756137479542
intercept: 5.633333333333329
slope: [0.54]
intercept: [5.63333333]
slope: [[0.54]]
predicted response:
[ 8.33333333 13.73333333 19.13333333 24.53333333 29.93333333 35.33333333]
predicted response:
[[ 8.33333333]
 [13.73333333]
 [19.13333333]
 [24.53333333]
 [29.93333333]
 [35.33333333]]
[[0]
 [1]
 [2]
 [3]
 [4]]
[5.63333333 6.17333333 6.71333333 7.25333333 7.79333333]
